In [1]:
import polars as pl
from typing import *
import sys
sys.path.append('../src')

# General Usage

In [2]:
import polars as pl
from dsds.utils import get_numpy
from dsds.prescreen import *
from dsds.transform import *
from dsds.fs import discrete_ig, f_classification, mrmr

In [3]:
df = pl.read_csv("../data/advertising.csv")
target = "Clicked on Ad"
df.head() 

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Constant,Test_Str_Constant,Test_BadColumn,Test_Binary
f64,i64,f64,f64,str,str,i64,str,str,i64,str,i64,f64,f64,i64,str,i64,str
68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""",0,"""Tunisia""","""3/27/2016 0:53…",0,"""A""",30,12.0,12.0,1,"""SSS""",0,"""A"""
80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""",1,"""Nauru""","""4/4/2016 1:39""",0,"""B""",30,9.0,13.0,1,"""SSS""",null,"""B"""
69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""",0,"""San Marino""","""3/13/2016 20:3…",0,"""A""",20,11.0,11.0,1,"""SSS""",0,"""A"""
74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…",1,"""Italy""","""1/10/2016 2:31…",0,"""B""",20,12.0,10.0,1,"""SSS""",null,"""B"""
68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""",0,"""Iceland""","""6/3/2016 3:36""",0,"""A""",30,11.0,14.0,1,"""SSS""",0,"""A"""


In [4]:
describe(df)

column,is_binary,count,null_count,null_pct,n_unique,unique_pct,mean,std,min,max,25%,median,75%,skew,kurtosis,dtype
str,i32,f64,f64,f64,u32,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str
"""Daily Time Spe…",0,1000.0,0.0,0.0,900,0.9,65.0002,15.853615,"""32.6""","""91.43""",51.38,68.215,78.57,-0.371203,-1.096058,"""numeric"""
"""Age""",0,1000.0,0.0,0.0,43,0.043,36.009,8.785562,"""19.0""","""61.0""",29.0,35.0,42.0,0.478423,-0.404518,"""numeric"""
"""Area Income""",0,1000.0,7.0,0.007,994,0.994,54980.756103,13434.996236,"""13996.5""","""79484.8""",47051.02,57009.76,65461.92,-0.648682,-0.110118,"""numeric"""
"""Daily Internet…",0,1000.0,11.0,0.011,957,0.957,180.024985,43.903142,"""104.78""","""269.96""",138.71,183.42,218.79,-0.04022,-1.277134,"""numeric"""
"""Ad Topic Line""",0,1000.0,0.0,0.0,1000,1.0,null,null,"""Adaptive 24hou…","""Visionary reci…",null,null,null,null,null,"""string"""
"""City""",0,1000.0,0.0,0.0,969,0.969,null,null,"""Adamsbury""","""Zacharyton""",null,null,null,null,null,"""string"""
"""Male""",1,1000.0,0.0,0.0,2,0.002,0.481,0.499889,"""0.0""","""1.0""",0.0,0.0,1.0,0.076055,-1.994216,"""numeric"""
"""Country""",0,1000.0,0.0,0.0,237,0.237,null,null,"""Afghanistan""","""Zimbabwe""",null,null,null,null,null,"""string"""
"""Timestamp""",0,1000.0,0.0,0.0,997,0.997,null,null,"""1/1/2016 15:14…","""7/9/2016 16:23…",null,null,null,null,null,"""string"""


In [5]:
describe_str(df, words_to_count=["A", "Visionary"])

features,null_count,null_pct,min,max,mode,min_byte_len,max_byte_len,avg_byte_len,median_byte_len,avg_space_cnt,avg_digit_cnt,avg_cap_cnt,avg_lower_cnt,total_A_count,total_Visionary_count
str,u32,f64,str,str,str,u32,u32,f64,f64,f64,f64,f64,f64,u32,u32
"""Ad Topic Line""",0,0.0,"""Adaptive 24hou…","""Visionary reci…","""Seamless impac…",17,55,33.394,33.0,2.219,0.087,1.108,29.221,60,7
"""City""",0,0.0,"""Adamsbury""","""Zacharyton""","""Lisamouth""",5,23,11.887,11.0,0.471,0.0,1.471,9.945,65,0
"""Country""",0,0.0,"""Afghanistan""","""Zimbabwe""","""France""",4,51,10.309,8.0,0.494,0.006,1.424,8.328,107,0
"""Timestamp""",0,0.0,"""1/1/2016 15:14…","""7/9/2016 16:23…","""5/30/2016 8:02…",13,15,14.273,14.0,1.0,10.273,0.0,0.0,0,0
"""One_Hot_Test""",0,0.0,"""A""","""C""","""A""",1,1,1.0,1.0,0.0,0.0,1.0,0.0,783,0
"""Test_Str_Const…",0,0.0,"""SSS""","""SSS""","""SSS""",3,3,3.0,3.0,0.0,0.0,3.0,0.0,0,0
"""Test_Binary""",0,0.0,"""A""","""B""","""A""",1,1,1.0,1.0,0.0,0.0,1.0,0.0,519,0


In [6]:
df = date_removal(df)

INFO:dsds.prescreen:Date Inferral is error prone due to the huge variety of date formats. Please use with caution.
INFO:dsds.prescreen:The following columns are dropped because they are dates. ['Timestamp'].
Removed a total of 1 columns.


In [7]:
df.head()

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Constant,Test_Str_Constant,Test_BadColumn,Test_Binary
f64,i64,f64,f64,str,str,i64,str,i64,str,i64,f64,f64,i64,str,i64,str
68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""",0,"""Tunisia""",0,"""A""",30,12.0,12.0,1,"""SSS""",0,"""A"""
80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""",1,"""Nauru""",0,"""B""",30,9.0,13.0,1,"""SSS""",null,"""B"""
69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""",0,"""San Marino""",0,"""A""",20,11.0,11.0,1,"""SSS""",0,"""A"""
74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…",1,"""Italy""",0,"""B""",20,12.0,10.0,1,"""SSS""",null,"""B"""
68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""",0,"""Iceland""",0,"""A""",30,11.0,14.0,1,"""SSS""",0,"""A"""


In [8]:
df = var_removal(df, threshold=0.5, target=target)

INFO:dsds.prescreen:The following columns are dropped because they have lower than 0.5 variance. ['Male', 'Test_Constant', 'Test_BadColumn'].
Removed a total of 3 columns.


In [9]:
df = constant_removal(df)

INFO:dsds.prescreen:The following columns are dropped because they are constants. ['Test_Str_Constant'].
Removed a total of 1 columns.


In [10]:
df = binary_encode(df, exclude = [target])

In [11]:
df.head()

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary
f64,i64,f64,f64,str,str,str,i64,str,i64,f64,f64,u8
68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12.0,12.0,0
80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9.0,13.0,1
69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11.0,11.0,0
74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12.0,10.0,1
68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11.0,14.0,0


In [12]:
# Second way (imo, better) to interactive with TransformationResult objects
df = impute(df, ["Area Income"], "mean")
df = impute(df, ["Daily Internet Usage", "Daily Internet Usage Band", "Area Income Band"], "median")
df 

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary
f64,i64,f64,f64,str,str,str,i64,str,i64,f64,f64,u8
68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12.0,12.0,0
80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9.0,13.0,1
69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11.0,11.0,0
74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12.0,10.0,1
68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11.0,14.0,0
59.99,23,59761.56,226.74,"""Sharable clien…","""Jamieberg""","""Norway""",0,"""B""",20,11.0,11.0,1
88.91,33,54980.756103,208.36,"""Enhanced dedic…","""Brandonstad""","""Myanmar""",0,"""A""",30,10.0,11.0,0
66.0,48,24593.33,131.76,"""Reactive local…","""Port Jefferybu…","""Australia""",1,"""A""",40,6.0,4.0,1
74.53,30,68862.0,221.51,"""Configurable c…","""West Colin""","""Grenada""",0,"""A""",30,11.0,13.0,1


In [13]:
df = scale(df, ["Area Income", "Daily Internet Usage"], "standard")
df 

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Binary
f64,i64,f64,f64,str,str,str,i64,str,i64,f64,f64,u8
68.95,35,0.511893,1.74127,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""",0,"""A""",30,12.0,12.0,0
80.23,31,1.005471,0.313948,"""Monitored nati…","""West Jodi""","""Nauru""",0,"""B""",30,9.0,13.0,1
69.47,26,0.358921,1.292598,"""Organic bottom…","""Davidton""","""San Marino""",0,"""A""",20,11.0,11.0,0
74.15,29,-0.01304,1.507658,"""Triple-buffere…","""West Terrifurt…","""Italy""",0,"""B""",20,12.0,10.0,1
68.37,35,1.412418,1.042496,"""Robust logisti…","""South Manuel""","""Iceland""",0,"""A""",30,11.0,14.0,0
59.99,23,0.3571,1.069064,"""Sharable clien…","""Jamieberg""","""Norway""",0,"""B""",20,11.0,11.0,1
88.91,33,-1.0869e-15,0.648105,"""Enhanced dedic…","""Brandonstad""","""Myanmar""",0,"""A""",30,10.0,11.0,0
66.0,48,-2.269777,-1.106273,"""Reactive local…","""Port Jefferybu…","""Australia""",1,"""A""",40,6.0,4.0,1
74.53,30,1.036854,0.94928,"""Configurable c…","""West Colin""","""Grenada""",0,"""A""",30,11.0,13.0,1


In [14]:
# Some of these are numerical columns, but we can still treat them as "discrete" columns
# and feed them into the information gain algorithm.
discrete = ["Age Band", "Country", "Area Income Band", "Daily Internet Usage Band", "Test_Binary", "One_Hot_Test"]
target = "Clicked on Ad"

In [15]:
discrete_ig(df, target, discrete_cols=discrete)

100%|██████████| 6/6 [00:00<00:00, 272.67it/s]


feature,target_entropy,conditional_entropy,unique_pct,information_gain,weighted_information_gain
str,f64,f64,f64,f64,f64
"""Age Band""",0.693147,0.569156,0.006,0.123991,0.123247
"""Country""",0.693147,0.558555,0.237,0.134592,0.102694
"""Area Income Ba…",0.693147,0.553505,0.014,0.139642,0.137687
"""Daily Internet…",0.693147,0.272341,0.009,0.420806,0.417019
"""Test_Binary""",0.693147,0.692424,0.002,0.000723,0.000722
"""One_Hot_Test""",0.693147,0.692173,0.003,0.000975,0.000972


In [16]:
# If nothing is given, automatically infers discrete columns. (See the docstring of discrete_inferral)
discrete_ig(df, target)

100%|██████████| 6/6 [00:00<00:00, 285.66it/s]


feature,target_entropy,conditional_entropy,unique_pct,information_gain,weighted_information_gain
str,f64,f64,f64,f64,f64
"""Age""",0.693147,0.532225,0.043,0.160922,0.154002
"""One_Hot_Test""",0.693147,0.692173,0.003,0.000975,0.000972
"""Age Band""",0.693147,0.569156,0.006,0.123991,0.123247
"""Daily Internet…",0.693147,0.272341,0.009,0.420806,0.417019
"""Area Income Ba…",0.693147,0.553505,0.014,0.139642,0.137687
"""Test_Binary""",0.693147,0.692424,0.002,0.000723,0.000722


In [17]:
# df["Ad Topic Line"].unique() # has 1000 uniques. 
# There is no randomness at all because we can say that there are two distinct subsets that perfects differentiates 0s from 1s

In [18]:
f_classification(df, target=target)

feature,f_value,p_value
str,f64,f64
"""Daily Time Spe…",1268.525161,5.8774e-180
"""Age""",319.643165,3.1298e-62
"""Area Income""",288.179499,5.6266e-57
"""Daily Internet…",1567.672992,7.6499e-207
"""Age Band""",269.875798,7.3704e-54
"""Daily Internet…",1528.282249,1.7330e-203
"""Area Income Ba…",285.144765,1.8363e-56
"""Test_Binary""",1.445286,0.229571


In [19]:
nums = get_numeric_cols(df, exclude=[target])
nums 

['Daily Time Spent on Site',
 'Age',
 'Area Income',
 'Daily Internet Usage',
 'Age Band',
 'Daily Internet Usage Band',
 'Area Income Band',
 'Test_Binary']

In [20]:
from sklearn.feature_selection import f_classif
 
f, pv = f_classif(df.select(nums), df[target])
pl.from_records([nums, f, pv], schema=["feature", "f_value", "p_value"])

feature,f_value,p_value
str,f64,f64
"""Daily Time Spe…",1268.525161,5.8774e-180
"""Age""",319.643165,3.1298e-62
"""Area Income""",288.179499,5.6266e-57
"""Daily Internet…",1567.672992,7.6499e-207
"""Age Band""",269.875798,7.3704e-54
"""Daily Internet…",1528.282249,1.7330e-203
"""Area Income Ba…",285.144765,1.8363e-56
"""Test_Binary""",1.445286,0.229571


In [21]:
from sklearn.feature_selection import f_regression

result = f_regression(df[nums].to_numpy(), df[target].to_numpy())
pl.from_records([nums, result[0], result[1]], schema=["feature", "f_value", "p_value"])

feature,f_value,p_value
str,f64,f64
"""Daily Time Spe…",1268.525161,5.8774e-180
"""Age""",319.643165,3.1298e-62
"""Area Income""",288.179499,5.6266e-57
"""Daily Internet…",1567.672992,7.6499e-207
"""Age Band""",269.875798,7.3704e-54
"""Daily Internet…",1528.282249,1.7330e-203
"""Area Income Ba…",285.144765,1.8363e-56
"""Test_Binary""",1.445286,0.229571


In [22]:
df_test = pl.concat([df.clone()] * 500)
df_test.shape

(500000, 13)

In [23]:
%%timeit 
f_classification(df_test, target="Clicked on Ad", num_cols=nums)

6.3 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit 
f, pv = f_classif(df_test[nums].to_numpy(), df_test[target].to_numpy())
pl.from_records([nums, f, pv], schema=["feature", "f_value", "p_value"])

59.6 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
result = f_regression(df_test[nums].to_numpy(), df_test[target].to_numpy())
pl.from_records([nums, result[0], result[1]], schema=["feature", "f_value", "p_value"])

18.8 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
del df_test 

In [27]:
# MRMR Method 
mrmr(df, target, 5)

AttributeError: 'str' object has no attribute 'info'

In [ ]:
df = one_hot_encode(df, cols=["One_Hot_Test"])

In [ ]:
df.head()

In [ ]:
df = ordinal_auto_encode(df, cols=["City", "Country"])

In [ ]:
df.head()

In [ ]:
final_df = remove_if_exists(df, ["Ad Topic Line", "Timestamp"]) # Timestamp was removed at the beginning
final_df.head() 

In [ ]:
np_data = get_numpy(final_df, target)
X, y, features = np_data.X, np_data.y, np_data.features

In [ ]:
X.shape 

In [ ]:
y.shape 

In [ ]:
features